In [ ]:


### Ścieżki wejścia/wyjścia dla modułu "Quality Metrics"

from pathlib import Path

PROJECT_ROOT = Path(".").resolve()

# katalog z obrazami
IMAGE_ROOT = PROJECT_ROOT / "inputs"   # jeżeli masz inną ścieżkę, zostaw swoją
print("IMAGE_ROOT:", IMAGE_ROOT, "| istnieje:", IMAGE_ROOT.exists())

# katalog artefaktów jakości (opcjonalny: miniatury, wykresy)
OUTPUT_DIR = PROJECT_ROOT / "outputs" / "quality"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# wspólny katalog na CSV całego pipeline'u
OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# finalny CSV z metrykami jakości
QUALITY_CSV = OUTPUT_CSV_DIR / "quality_metrics.csv"

print("QUALITY_CSV:", QUALITY_CSV)

### 01 – Importy + ścieżki + lista obrazów

In [1]:
# %%
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image

DIR_DATA = Path("data/test_scenes")

def list_images(dir_path: Path):
    exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
    return sorted([p for p in dir_path.iterdir() if p.suffix.lower() in exts])

image_paths = list_images(DIR_DATA)
print("Liczba obrazów:", len(image_paths))

Liczba obrazów: 271


### 02 – Metryki jakości

In [2]:
# %%
def load_cv2_gray(path: Path):
    return cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

def blur_score(img_gray):
    return float(cv2.Laplacian(img_gray, cv2.CV_64F).var())

def contrast_score(img_gray):
    return float(img_gray.std())

def brightness_score(img_gray):
    return float(img_gray.mean())

def compute_quality(path: Path):
    img_gray = load_cv2_gray(path)
    if img_gray is None:
        return None
    return {
        "file_path": str(path),
        "blur": blur_score(img_gray),
        "contrast": contrast_score(img_gray),
        "brightness": brightness_score(img_gray),
    }

### 03 – Batch

In [ ]:
# %%
records = []

for i, p in enumerate(image_paths, start=1):
    print(f"[{i}/{len(image_paths)}] {p.name}")
    rec = compute_quality(p)
    if rec:
        records.append(rec)

df_quality = pd.DataFrame(records)
df_quality



[1/271] JMi_0014.jpg
[2/271] JMi_0127.tif
[3/271] JMi_0128.tif
[4/271] JMi_0129.tif
[5/271] JMi_0131.tif
[6/271] JMi_0175.jpg
[7/271] JMi_0177.jpg
[8/271] JMi_0192.tif
[9/271] JMi_0339.tif
[10/271] JMi_0342.tif
[11/271] JMi_0344.tif
[12/271] JMi_0348.tif
[13/271] JMi_0349.tif
[14/271] JMi_0350.tif
[15/271] JMi_0356.tif
[16/271] JMi_0359.jpg
[17/271] JMi_0360.tif
[18/271] JMi_0363.tif
[19/271] JMi_0365.jpg
[20/271] JMi_0366.jpg
[21/271] JMi_0368.tif
[22/271] JMi_0369.tif
[23/271] JMi_0370.tif
[24/271] JMi_0419.tif
[25/271] JMi_0421.jpg
[26/271] JMi_0424.tif
[27/271] JMi_0425.tif
[28/271] JMi_0426.tif
[29/271] JMi_0431.tif
[30/271] JMi_0433.tif
[31/271] JMi_0434.tif
[32/271] JMi_0436.jpg
[33/271] JMi_0464.tif
[34/271] JMi_0714.jpg
[35/271] JMi_0716.tif
[36/271] JMi_0730.jpg
[37/271] JMi_0744.jpg
[38/271] JMi_0745.jpg
[39/271] JMi_0750.tif
[40/271] JMi_0751.tif
[41/271] JMi_0752.tif
[42/271] JMi_0757.jpg
[43/271] JMi_0767.jpg
[44/271] JMi_0772.tif
[45/271] JMi_0802.jpg
[46/271] JMi_0803.j

,file_path,blur,contrast,brightness
0,data/test_scenes/JMi_0014.jpg,70.021233,40.994570,72.710079
1,data/test_scenes/JMi_0127.tif,559.242330,66.818501,123.475445
2,data/test_scenes/JMi_0128.tif,893.276208,63.361484,115.347440
3,data/test_scenes/JMi_0129.tif,3001.441202,63.125097,121.615615
4,data/test_scenes/JMi_0131.tif,285.559831,63.768107,125.127048
...,...,...,...,...
266,data/test_scenes/turzanskiw_914.tif,81.082730,65.671551,156.353747
267,data/test_scenes/turzanskiw_915.tif,176.241019,69.524651,150.930343
268,data/test_scenes/turzanskiw_918.tif,80.516354,62.549991,155.152460
269,data/test_scenes/turzanskiw_925k.tiff,101.539475,74.158225,130.375968


### CSV

In [ ]:
# %%
out_path = Path("outputs/quality_metrics.csv")
df_quality.to_csv(QUALITY_CSV, index=False)
print("Zapisano metryki jakości do:", QUALITY_CSV)

Zapisano: outputs/quality_metrics.csv


### 1. Sortowanie zdjęć po ostrości (blur)

In [6]:
# %%
"""
Sortowanie zdjęć po ostrości (blur).

Tworzy dwie tabele:
- df_sharpest  – zdjęcia od najostrzejszych
- df_blurriest – zdjęcia od najbardziej rozmytych
"""

df_sharpest = df_quality.sort_values("blur", ascending=False).reset_index(drop=True)
df_blurriest = df_quality.sort_values("blur", ascending=True).reset_index(drop=True)

print("Najostrzejsze 10 zdjęć:")
df_sharpest.head(10)

Najostrzejsze 10 zdjęć:


,file_path,blur,contrast,brightness
0,data/test_scenes/JMi_7707.tif,3821.602628,82.495026,116.938032
1,data/test_scenes/JMi_7705.tif,3704.879292,84.030299,118.465011
2,data/test_scenes/JMi_0129.tif,3001.441202,63.125097,121.615615
3,data/test_scenes/JMi_4044.jpg,2904.440328,69.619136,96.966802
4,data/test_scenes/JMi_6714.tif,2631.915480,55.740048,126.227364
5,data/test_scenes/JMi_0177.jpg,2452.938418,75.046985,106.190990
6,data/test_scenes/JMi_6717.tif,2128.127824,57.698873,145.513151
7,data/test_scenes/JMi_0360.tif,1985.148851,58.243679,80.534233
8,data/test_scenes/JMi_6775.tif,1677.125659,64.533714,121.690652
9,data/test_scenes/JMi_6709.tif,1649.630960,70.481623,126.738289


In [7]:
# %%
print("Najbardziej rozmyte 10 zdjęć:")
df_blurriest.head(10)

Najbardziej rozmyte 10 zdjęć:


,file_path,blur,contrast,brightness
0,data/test_scenes/JMi_5821.tif,6.489652,74.988280,132.063803
1,data/test_scenes/turzanskiw_579.tif,8.690249,57.502111,141.703086
2,data/test_scenes/JMi_0421.jpg,9.038213,47.753339,81.359812
3,data/test_scenes/turzanskiw_585.tif,9.844245,59.594530,145.265561
4,data/test_scenes/turzanskiw_574.tif,9.893877,52.495802,159.156134
5,data/test_scenes/turzanskiw_578.tif,10.052350,58.820089,100.651689
6,data/test_scenes/turzanskiw_588.tif,10.494082,59.354131,145.821896
7,data/test_scenes/turzanskiw_571.tif,10.591559,57.631161,133.905259
8,data/test_scenes/turzanskiw_757.tif,10.738632,84.018725,133.507600
9,data/test_scenes/turzanskiw_573.tif,10.873384,59.091726,144.600572


### 2. Filtrowanie „najgorszych” jakościowo (np. 10% najbardziej rozmytych)

In [8]:
# %%
"""
Wybór najgorszych jakościowo zdjęć pod względem rozmycia.

Wyznacza próg jako 10. percentyl wartości 'blur'
i zwraca wszystkie zdjęcia poniżej tego progu.
"""

blur_threshold = df_quality["blur"].quantile(0.10)
print("Próg blur (10% najbardziej rozmyte):", blur_threshold)

df_worst_blur = df_quality[df_quality["blur"] <= blur_threshold].copy()
df_worst_blur = df_worst_blur.sort_values("blur", ascending=True).reset_index(drop=True)

print("Liczba zdjęć uznanych za najbardziej rozmyte:", len(df_worst_blur))
df_worst_blur

Próg blur (10% najbardziej rozmyte): 48.06209443700113
Liczba zdjęć uznanych za najbardziej rozmyte: 28


,file_path,blur,contrast,brightness
0,data/test_scenes/JMi_5821.tif,6.489652,74.988280,132.063803
1,data/test_scenes/turzanskiw_579.tif,8.690249,57.502111,141.703086
2,data/test_scenes/JMi_0421.jpg,9.038213,47.753339,81.359812
3,data/test_scenes/turzanskiw_585.tif,9.844245,59.594530,145.265561
4,data/test_scenes/turzanskiw_574.tif,9.893877,52.495802,159.156134
5,data/test_scenes/turzanskiw_578.tif,10.052350,58.820089,100.651689
6,data/test_scenes/turzanskiw_588.tif,10.494082,59.354131,145.821896
7,data/test_scenes/turzanskiw_571.tif,10.591559,57.631161,133.905259
8,data/test_scenes/turzanskiw_757.tif,10.738632,84.018725,133.507600
9,data/test_scenes/turzanskiw_573.tif,10.873384,59.091726,144.600572
